# Modèle multivarié des K plus proches voisins

## Récapitulatif

+ Utiliser la méthode DataFrame.info() pour retourner le nombre de valeurs non nulles de chaque colonne.

In [4]:
import pandas as pd
import numpy as np
np.random.seed(1)
import warnings
warnings.filterwarnings('ignore')

paris_listings = pd.read_csv('paris_airbnb.csv')
paris_listings = paris_listings.loc[np.random.permutation(len(paris_listings))]
stripped_commas = paris_listings['price'].str.replace(',', '')
stripped_dollars = stripped_commas.str.replace('$', '')
paris_listings['price'] = stripped_dollars.astype('float')

In [5]:
paris_listings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 4740 to 5157
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   host_response_rate    5000 non-null   object 
 1   host_acceptance_rate  0 non-null      float64
 2   host_listings_count   7999 non-null   float64
 3   latitude              8000 non-null   float64
 4   longitude             8000 non-null   float64
 5   city                  7997 non-null   object 
 6   zipcode               7930 non-null   object 
 7   state                 7977 non-null   object 
 8   accommodates          8000 non-null   int64  
 9   room_type             8000 non-null   object 
 10  bedrooms              7976 non-null   float64
 11  bathrooms             7942 non-null   float64
 12  beds                  7986 non-null   float64
 13  price                 8000 non-null   float64
 14  cleaning_fee          6250 non-null   object 
 15  security_deposit  

## Supprimer des caractéristiques

+ Supprimer les 9 colonnes que nous venons de citer du DataFrame paris_listings:
 - 3 contenant des valeurs non numériques (room_type, city et state)
 - 3 contenant des valeurs numériques mais non ordinales (longitude, latitude et zipcode)
 - 3 décrivant l'hôte plutôt que le logement (host_)
+ Afficher la somme des valeurs manquantes du DataFrame résultant.

In [6]:
paris_listings.isna().sum()

host_response_rate      3000
host_acceptance_rate    8000
host_listings_count        1
latitude                   0
longitude                  0
city                       3
zipcode                   70
state                     23
accommodates               0
room_type                  0
bedrooms                  24
bathrooms                 58
beds                      14
price                      0
cleaning_fee            1750
security_deposit        1680
minimum_nights             0
maximum_nights             0
number_of_reviews          0
dtype: int64

In [7]:
drop_columns = ['room_type', 'city', 'state', 'longitude', 'latitude', 'zipcode', 'host_response_rate', 'host_acceptance_rate', 'host_listings_count']
paris_listings = paris_listings.drop(drop_columns, axis=1)
print(paris_listings.isna().sum())

accommodates            0
bedrooms               24
bathrooms              58
beds                   14
price                   0
cleaning_fee         1750
security_deposit     1680
minimum_nights          0
maximum_nights          0
number_of_reviews       0
dtype: int64


## Gérer les valeurs manquantes

+ Supprimer les colonnes cleaning_fee et security_deposit du DataFrame paris_listings.
+ Ensuite, supprimer toutes les lignes contenant une valeur manquante dans les colonnes 'bedrooms', 'bathrooms' et 'beds' du dataframe paris_listings.
 - Vous pouvez accomplir cela en utilisant la méthode dropna() en fixant le paramètre axis à 0.
 - Etant donné que seules les colonnes bedrooms, bathrooms, et beds contiennent des valeurs manquantes, les lignes contenant ces valeurs manquantes dans ces colonnes seront supprimées.
+ Afficher le nombre de valeurs nulles pour le nouveau DataFrame nouvellement mis à jour pour confirmer qu'il n'y a plus aucune valeur manquante.


In [8]:
paris_listings = paris_listings.drop(['cleaning_fee', 'security_deposit'], axis=1)
paris_listings = paris_listings.dropna(axis=0)
print(paris_listings.isnull().sum())

accommodates         0
bedrooms             0
bathrooms            0
beds                 0
price                0
minimum_nights       0
maximum_nights       0
number_of_reviews    0
dtype: int64


## Normaliser les colonnes

+ Normaliser toutes les colonnes restantes de paris_listings et assigner le nouveau DataFrame contenant juste les colonnes normalisées à la variable normalized_listings.
+ Ajouter la colonne 'price' de paris_listings à normalized_listings.
+ Afficher les 3 premières valeurs de normalized_listings.


In [11]:
normalized_listings = (paris_listings - paris_listings.mean())/(paris_listings.std())
normalized_listings['price'] = paris_listings['price']
normalized_listings.head(3)

,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
4740,0.503868,-0.296884,-0.293537,0.20531,65.0,-0.184601,1.062858,-0.564545
5606,-0.131849,0.892605,0.843973,0.20531,98.0,-0.101183,1.061018,-0.636924
4824,-0.767566,-1.486372,-0.293537,-0.64526,65.0,0.037847,1.062858,-0.651400


## Distance euclidienne pour le cas multivarié

+ Calculer la distance Euclidienne en utilisant seulement les caractéristiques accommodates et bedrooms entre la première ligne et la 5e ligne de normalized_listings en utilisant la fonction distance.euclidean().
+ Assigner la valeur de distance à la variable first_fifth_distance et afficher le résultat.


In [12]:
from scipy.spatial import distance
first_listing = normalized_listings.iloc[0][['accommodates', 'bedrooms']]
fifth_listing = normalized_listings.iloc[4][['accommodates', 'bedrooms']]
first_fifth_distance = distance.euclidean(first_listing, fifth_listing)

print(first_fifth_distance)

0.6357172321498359


## Adapter un modèle et faire des prédictions

+ Créer une instance de la classe **KNeighborsRegressor** avec les paramètres suivants:
 - n_neighbors: 5
 - algorithm: brute
+ Utiliser la méthode **fit** pour spécifier les données que nous voulons utiliser pour notre modèle des k plus proches voisins. Utiliser les paramètres suivants:
 - Les données d'entrainement, les colonnes de caractéristiques: seulement les colonnes 'accommodates' et 'bedrooms', dans cet ordre, depuis le DataFrame train_df.
 - La colonne cible: la colonne 'price' du DataFrame train_df.
+ Appeler la méthode **predict** pour faire des prédictions dessus:
 - Les colonnes 'accommodates' et 'bedrooms' du DataFrame test_df
 - Assigner le tableau Numpy résultant des valeurs de prix prédits à la variable predictions.


In [15]:
from sklearn.neighbors import KNeighborsRegressor

train_df = normalized_listings.iloc[0:6000]
test_df = normalized_listings.iloc[6000:]
train_columns = ['accommodates', 'bedrooms']

# instancier le modèle de ML
knn = KNeighborsRegressor(n_neighbors=5, algorithm='brute')

# adapter le modèle aux données
knn.fit(train_df[train_columns], train_df['price'])

# utiliser le modèle pour faire des prédictions
predictions = knn.predict(test_df[train_columns])

predictions

array([ 72. ,  78.2,  78.2, ...,  72. , 148.4,  78.2])

## Calculer l'erreur quadratique moyenne

+ Utiliser la fonction mean_squared_error pour calculer la valeur d'erreur quadratique moyenne MSE pour les prédictions que nous avons faites.
+ Assigner la valeur de MSE à la variable two_features_mse.
+ Calculer la valeur de la racine carrée de l'erreur quadratique moyenne RMSE en prenant la racine carrée de la valeur MSE et assigner le résultat à la variable two_features_rmse.
+ Afficher ces 2 résultats d'erreur.


In [16]:
from sklearn.metrics import mean_squared_error

two_features_mse = mean_squared_error(test_df['price'], predictions)
two_features_rmse = two_features_mse**(1/2)

print(two_features_mse)
print(two_features_rmse)

5894.320834637454
76.77448036058242


## Utiliser plus de caractéristiques

+ Créer une nouvelle instance de la classe KNeighborsRegressor avec les paramètres suivants:
+ Adapter le modèle pour qu'il utilise les colonnes suivantes de notre set d'entrainement (train_df):
 - accommodates
 - bedrooms
 - bathrooms
 - number_of_reviews
+ Utiliser le modèle pour faire des prédictions sur le set de test (test_df) en utilisant les mêmes colonnes. Assigner le tableau Numpy des prédictions à la variable four_predictions.
+ Utiliser la fonction mean_squared_error() pour calculer la valeur MSE pour ces prédictions en comparant les valeurs de four_predictions avec la colonne price du DataFrame test_df. Assigner la valeur MSE calculée à la variable four_mse.
+ Calculer la valeur RMSE et assigner le résultat à la variable four_rmse.
+ Afficher les résultats four_mse et four_rmse.


In [17]:
features = ['accommodates', 'bedrooms', 'bathrooms', 'number_of_reviews']
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=5, algorithm='brute')
knn.fit(train_df[features], train_df['price'])
four_predictions = knn.predict(test_df[features])
four_mse = mean_squared_error(test_df['price'], four_predictions)
four_rmse = four_mse**(1/2)

print(four_mse)
print(four_rmse)

5424.405821596245
73.65056565700121


## Utiliser toutes les caractéristiques

+ Utiliser toutes les colonnes, à l'exception de la colonne 'price', pour entrainer notre modèle des k plus proches voisins en utilisant les mêmes paramètres pour la classe KNeighborsRegressor que les précédentes vidéos.
+ Utiliser le modèle pour faire les prédictions sur le set de test et assigner le tableau Numpy résultant des prédictions à la variable all_features_predictions.
+ Calculer les valeurs MSE et RMSE et assigner les résultats aux variables all_features_mse et all_features_rmse.
+ Afficher les scores d'erreurs.


In [18]:
knn = KNeighborsRegressor(n_neighbors=5, algorithm='brute')

features = train_df.columns.tolist()
features.remove('price')

knn.fit(train_df[features], train_df['price'])
all_features_predictions = knn.predict(test_df[features])

all_features_mse = mean_squared_error(test_df['price'], all_features_predictions)
all_features_rmse = all_features_mse**(1/2)

print(all_features_mse)
print(all_features_rmse)

7227.205279081899
85.01297124016958
